In [1]:
from ultralytics import YOLO
import cv2

# Load your custom-trained YOLO model
model = YOLO('../training_results/first_run/weights/best.pt')
model.conf = 0.5
# Open the video
video_path = "../input/video/right_profile_1.mov"
cap = cv2.VideoCapture(video_path)

# prepare video writer for saving results
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
output_path = '../output/video/output_video.avi'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (width, height))

# Process video
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
        
    # Run inference on the frame
    results = model.track(frame, persist=True, verbose=False)
    
    # Visualize the results on the frame
    # If any detections exist, keep only the highest confidence one
    if len(results[0].boxes) > 0:
        # Sort by confidence and keep only the highest
        boxes = results[0].boxes
        conf_values = boxes.conf.cpu().numpy()
        highest_conf_index = conf_values.argmax()
        conf = conf_values[highest_conf_index]
        
        # Get just the highest confidence box
        single_box = boxes[highest_conf_index].xyxy.cpu().numpy()[0]
        
        # Draw only this box
        x1, y1, x2, y2 = map(int, single_box[:4])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        conf_text = f"Conf: {conf:.2f}"
        cv2.putText(frame, conf_text, (x1, y1-10), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    # Write the frame with the detections to the output video
    
    rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    out.write(rotated_frame)
    
    # Display the annotated frame (optional)
    cv2.imshow("YOLOv8 Inference", rotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

2025-04-08 14:05:56.786 Python[61165:7201766] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-08 14:05:56.786 Python[61165:7201766] +[IMKInputSession subclass]: chose IMKInputSession_Modern
